In [1]:
%matplotlib inline


Implementing FAQ bot with a Sequence to Sequence Network using Attention Mechanism
************************************************************************************
**Source**: `https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py>`_

In this project we will be training a neural network to predict Answer of FAQ's asked by users from
Amazon AWS EC2 FAQ

::

    -- > input, = target, < output

    > what can developers now do that they could not before
    = developers are now free to innovate knowing that no matter how successful their businesses become it will be      inexpensive and simple to ensure they have the compute capacity they need to meet their business requirements .
    < developers are now free to innovate knowing that no matter how successful their businesses become it will be inexpensive and simple to ensure they have the compute capacity they need to meet their business requirements . 

    

**Requirements**

- Jupyter IPython Installed
- Download and Install Version 0.4 of Pytorch
- Training Data txt file included along with IPython Notebook



In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import bcolz  # to process the data from Glove File 
import pickle # to dump and load pretrained glove vectors 
import copy   # to make deepcopy of python lists and dictionaries
import operator
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading Glove Word Vectors

Initially using 50 dimensional vectors

In [6]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'glove.6B/6B.50.dat', mode='w')

with open(f'glove.6B/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir=f'glove.6B/6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'glove.6B/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'glove.6B/6B.50_idx.pkl', 'wb'))

Using those objects we can now create a dictionary that given a word returns its vector.

In [7]:
vectors = bcolz.open(f'glove.6B/6B.50.dat')[:]
words = pickle.load(open(f'glove.6B/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'glove.6B/6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}


In [8]:
copy_of_words2idx = copy.deepcopy(word2idx)
copy_of_words = copy.deepcopy(words)

In [9]:
# for sos token - dictionary value swapping copy_of_words & copy_of_words2idx

tempword1 = copy_of_words[0]
copy_of_words[0] = copy_of_words[copy_of_words2idx['sos']]
copy_of_words[copy_of_words2idx['sos']] = tempword1

temp_word2idx1 = copy_of_words2idx['sos']
copy_of_words2idx['sos'] = 0 
copy_of_words2idx[words[0]] = temp_word2idx1

# for eos token - dictionary value swapping copy_of_words & copy_of_words2idx

tempword2 = copy_of_words[1]
copy_of_words[1] = copy_of_words[copy_of_words2idx['eos']]
copy_of_words[copy_of_words2idx['eos']] = tempword2

temp_word2idx2 = copy_of_words2idx['eos']
copy_of_words2idx['eos'] = 1 
copy_of_words2idx[words[1]] = temp_word2idx2


# Creating Sorted Instance of word2idx 

In [10]:
copy_of_words2idx = { k : v for k , v in sorted(copy_of_words2idx.items(), key=operator.itemgetter(1))}

Loading data files
==================

The data for this project is a set of Three Hundred and Fifty Question and Answer Pairs 
The file is a tab separated list of Question Answer Pairs:

::

    Amazon EC2s simple web service interface allows you to obtain and configure capacity with minimal friction. 	What can I do with Amazon EC2





each word in training set is represented as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word).





We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




# Creating 2 Seperate Classes for Input Lang and Ouput Lang

Class 1 - > InputLang - built from Glove 

Class 2 - > OutputLang - built from Target Dictionary 


In [11]:
var = 1
class InputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = { k : v for k , v in sorted(copy_of_words2idx.items(), key=operator.itemgetter(1))}
        self.word2count = { word : 1 for word in copy_of_words }
        self.index2word = { i : word for word, i in copy_of_words2idx.items() }
        self.n_words = 400001

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

In [12]:
SOS_token = 0
EOS_token = 1


class OutputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [13]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all Answer → Question, so if we
want to translate from Question → Answer ``reverse``
flag is there to reverse the pairs.




In [14]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = InputLang(lang2)
        output_lang = OutputLang(lang1)
    else:
        input_lang = InputLang(lang1)
        output_lang = OutputLang(lang2)

    return input_lang, output_lang, pairs

In [15]:
MAX_LENGTH = 60


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [16]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('ans', 'que', True)
print(random.choice(pairs))

Reading lines...
Read 13 sentence pairs
Trimmed to 13 sentence pairs
Counting words...
Counted words:
que 400002
ans 187
['are there any limitations in sending email from amazon ec instances', 'yes in order to maintain the quality of amazon ec addresses for sending email we enforce default limits on the amount of email that can be sent from ec accounts .']


We must build a matrix of weights that will be loaded into the PyTorch embedding layer. Its shape will be equal to:
# (dataset’s vocabulary length, word vectors dimension)

In [17]:
matrix_len = input_lang.n_words

weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(input_lang.word2index):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))
        
print(weights_matrix)

[[ 0.51387    -0.71619    -0.27022    ...  0.16832     0.57609
   0.046694  ]
 [-0.30899     0.14484     1.1682     ... -0.40967    -0.46512
  -0.29577   ]
 [ 0.15164     0.30177    -0.16763    ... -0.35652     0.016413
   0.10216   ]
 ...
 [ 0.072617   -0.51393     0.4728     ... -0.18907    -0.59021
   0.55559   ]
 [-0.527827    0.31407215 -0.13412424 ...  0.18257573  0.03883834
  -0.90683415]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


# New Encoder

In [18]:
class EncoderRNNNew(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EncoderRNNNew, self).__init__()
        self.hidden_size = embedding_dim

        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder` is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.


Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for NLP tasks consisting long Sentences.


With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.






In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the Answer .




Simple Decoder


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

We have not used the Decoder Model here, and straight away used Attension Decoder Model explained below




Attention Decoder
******************

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.


In [20]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

In [21]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit instability `
You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [22]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [23]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [24]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [26]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [27]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place we can actually
initialize a network and start training.



In [28]:
hidden_size = 50
encoder1 = EncoderRNNNew(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 30000, print_every=1000)

11m 21s (- 329m 33s) (1000 3%) 2.8719
22m 50s (- 319m 53s) (2000 6%) 1.6498
34m 13s (- 308m 2s) (3000 10%) 0.5688
45m 35s (- 296m 19s) (4000 13%) 0.1977
56m 44s (- 283m 42s) (5000 16%) 0.0912
68m 2s (- 272m 8s) (6000 20%) 0.0419
79m 12s (- 260m 16s) (7000 23%) 0.0222
90m 44s (- 249m 32s) (8000 26%) 0.0138
102m 1s (- 238m 4s) (9000 30%) 0.0089
113m 22s (- 226m 45s) (10000 33%) 0.0072
124m 43s (- 215m 25s) (11000 36%) 0.0061
136m 1s (- 204m 2s) (12000 40%) 0.0053
147m 14s (- 192m 32s) (13000 43%) 0.0048
158m 22s (- 181m 0s) (14000 46%) 0.0111
169m 44s (- 169m 44s) (15000 50%) 0.0044
180m 53s (- 158m 16s) (16000 53%) 0.0037
192m 11s (- 146m 58s) (17000 56%) 0.1743
203m 24s (- 135m 36s) (18000 60%) 0.1228
214m 37s (- 124m 15s) (19000 63%) 0.0068
226m 2s (- 113m 1s) (20000 66%) 0.0190
237m 10s (- 101m 38s) (21000 70%) 0.0044
248m 29s (- 90m 21s) (22000 73%) 0.0036
259m 43s (- 79m 2s) (23000 76%) 0.0031
271m 1s (- 67m 45s) (24000 80%) 0.0028
282m 9s (- 56m 25s) (25000 83%) 0.0025
293m 33s (-

In [29]:
evaluateRandomly(encoder1, attn_decoder1)

> what can i do with amazon ec 
= amazon ec s simple web service interface allows you to obtain and configure capacity with minimal friction .
< amazon ec s simple web service interface allows you to obtain and configure capacity with minimal friction . <EOS>

> why am i asked to verify my phone number when signing up for amazon ec 
= amazon ec registration requires you to have a valid phone number and email address on file with aws in case we ever need to contact you .
< amazon ec registration requires you to have a valid phone number and email address on file with aws in case we ever need to contact you . <EOS>

> how do i access my systems
= the runinstances call that initiates execution of your application stack will return a set of dns names one for each system that is being booted .
< the runinstances call that initiates execution of your application stack will return a set of dns names one for each system that is being booted . <EOS>

> how can i get started with amazon ec 
= to